In [3]:
import pandas as pd
import numpy as np

def process_data(file_content):
    # DataFrame 생성
    df = pd.read_csv(file_content, names=['TIME','BUTTON', 'X', 'Y','Z','PRESSURE_NORMAL'])
    
    # 연속된 0의 구간을 찾기
    zero_segments = []
    current_segment = []
    k_count= 0
    count = 0
    #print(df['BUTTON'])
    for i, value in enumerate(df['BUTTON']):
        #print(i)
        #print(type(value))
        if value == str(0) and i >=1:
            count += 1
            current_segment.append(i)
        elif i == 0:
            continue
        else:
            if count >= 600 :  # 600개 이상 연속된 0인 경우
                k_count += 1
                zero_segments.append(current_segment)
            count = 0
            current_segment = []
            
    print(k_count)
    print(count)
    #print(zero_segments)
    
    # 분할 지점 찾기
    split_points = []
    for segment in zero_segments:
        
        split_points.extend([segment[0], segment[-1]])
    
    # 데이터 분할
    result_dfs = []
    start = 0

    for i in range(0, len(split_points), 2):
        if i+1 >= len(split_points):
            break
            
        # 0이 연속된 구간을 제외한 데이터 저장
        if start < split_points[i]:
            segment_df = df.iloc[start:split_points[i]]
            if not segment_df.empty:
                result_dfs.append(segment_df)
        
        start = split_points[i+1] + 1
    
    # 마지막 구간 처리
    if start < len(df):
        segment_df = df.iloc[start:]
        if not segment_df.empty:
            result_dfs.append(segment_df)
    
    return result_dfs

# 데이터 처리
dfs = process_data('yyeepp (5).csv')
print(len(dfs))
# 결과 파일 저장
for i, df in enumerate(dfs, 1):
    if i ==1:
        df = df[1:-1]
        df.to_csv(f'C:filehome/split_data_{i}.csv', index=False)
        print(f'파일 {i}: {len(df)} 행')

    else:
        df.to_csv(f'filehome/split_data_{i}.csv', index=False)
        print(f'파일 {i}: {len(df)} 행')

3
385
4
파일 1: 835 행
파일 2: 660 행
파일 3: 725 행
파일 4: 2417 행


In [ ]:
import pandas as pd
import numpy as np

# 데이터 불러오기

for i in range(5):
    input_file_path = f'filehome/split_data_{i+1}.csv'
    output_file_path = f"filehome/yqp_processed_data{i+1}.csv"
    data = pd.read_csv(input_file_path)

    # 전체 작업 수행 시간 계산
    data['TIME_DIFF'] = data['TIME'] - (data['TIME'].iloc[0] + 1)
    total_time = data['TIME_DIFF'].iloc[-1]

    # 공중에서 움직인 시간 (BUTTON이 0일 때)
    air_time = data[data['BUTTON'] == 0]['TIME_DIFF'].count()

    # 종이 위에서 움직인 시간 (BUTTON이 1일 때)
    paper_time = data[data['BUTTON'] == 1]['TIME_DIFF'].count()

    # 시간 차분이 0인 경우를 NaN으로 처리하여 속도와 가속도를 안전하게 계산
    data['TIME_DIFF_DELTA'] = data['TIME_DIFF'].diff().replace(0, np.nan)
    data['DISTANCE'] = np.sqrt(data['X'].diff()**2 + data['Y'].diff()**2)
    data['SPEED'] = data['DISTANCE'] / data['TIME_DIFF_DELTA']

    # 가속도 및 가가속도 계산
    data['ACCELERATION'] = data['SPEED'].diff() / data['TIME_DIFF_DELTA']
    data['JERK'] = data['ACCELERATION'].diff() / data['TIME_DIFF_DELTA']

    # 무한대와 NaN 값을 안전하게 처리
    data['SPEED'] = data['SPEED'].replace([np.inf, -np.inf], np.nan)
    data['ACCELERATION'] = data['ACCELERATION'].replace([np.inf, -np.inf], np.nan)
    data['JERK'] = data['JERK'].replace([np.inf, -np.inf], np.nan)

    # 종이 위와 공중에서의 평균 속도, 가속도, 가가속도 다시 계산
    mean_speed_on_paper = data[data['BUTTON'] == 1]['SPEED'].mean()
    mean_speed_in_air = data[data['BUTTON'] == 0]['SPEED'].mean()
    mean_accel_on_paper = data[data['BUTTON'] == 1]['ACCELERATION'].mean()
    mean_accel_in_air = data[data['BUTTON'] == 0]['ACCELERATION'].mean()
    mean_jerk_on_paper = data[data['BUTTON'] == 1]['JERK'].mean()
    mean_jerk_in_air = data[data['BUTTON'] == 0]['JERK'].mean()

    # 압력 값의 평균 및 분산 계산
    pressure_mean = data['PRESSURE_NORMAL'].mean()
    pressure_variance = data['PRESSURE_NORMAL'].var()

    # 수정된 GMRT 계산 함수
    def gmrt(button_state, d=1):
        subset = data[data['BUTTON'] == button_state]
        n = len(subset)
        if n <= d:  # 샘플 포인트가 비교 간격보다 적은 경우 0 반환
            return 0
        # 각 점의 반경 계산 (원점 또는 기준점으로부터의 거리)
        radii = np.sqrt(subset['X']**2 + subset['Y']**2)
        # d 간격으로 거리 차이 계산
        distances = np.abs(radii.diff(periods=d).dropna())
        gmrt_value = (1 / (n - d)) * distances.sum()
        return gmrt_value

    # 수정된 GMRT 값 계산
    gmrt_on_paper = gmrt(1, d=1)  # d 값을 1로 설정 (필요에 따라 다른 값으로 조정 가능)
    gmrt_in_air = gmrt(0, d=1)
    mean_gmrt = (gmrt_on_paper + gmrt_in_air) / 2

    # 펜이 종이에 닿은 횟수 (BUTTON이 0에서 1로 변할 때)
    pendowns = data['BUTTON'].diff().eq(1).sum()

    # X, Y 최대 확장
    x_extension = data['X'].max()
    y_extension = data['Y'].max()

    # 결과를 요청한 순서대로 딕셔너리로 정리
    result = {
        'air_time': air_time,
        'gmrt_in_air': gmrt_in_air,
        'gmrt_on_paper': gmrt_on_paper,
        'max_x_extension': x_extension,
        'max_y_extension': y_extension,
        'mean_acc_in_air': mean_accel_in_air,
        'mean_acc_on_paper': mean_accel_on_paper,
        'mean_gmrt': mean_gmrt,
        'mean_jerk_in_air': mean_jerk_in_air,
        'mean_jerk_on_paper': mean_jerk_on_paper,
        'mean_speed_in_air': mean_speed_in_air,
        'mean_speed_on_paper': mean_speed_on_paper,
        'num_of_pendown': pendowns,
        'paper_time': paper_time,
        'pressure_mean': pressure_mean,
        'pressure_var': pressure_variance,
        'total_time': total_time
    }

    # 결과를 데이터프레임으로 변환 후 CSV 파일로 저장
    result_df = pd.DataFrame([result])
    result_df.to_csv(output_file_path, index=False)


In [ ]:
import pandas as pd
import numpy as np
import joblib


for i in range(5):
    mnum= i+1

    # 데이터 로드
    data = pd.read_csv(f'filehome/yqp_processed_data{i+1}.csv')
    data.columns = data.columns + str(mnum)
    # 모델과 스케일러 로드
    rf_model = joblib.load(f'displacement_prediction_model{i+1}.joblib')
    scaler = joblib.load(f'displacement_scaler{i+1}.joblib')

    # 예측
    new_data_scaled = scaler.transform(data)
    predicted_disp_index1 = rf_model.predict(new_data_scaled)

    # 예측값을 데이터프레임의 두 번째 열에 추가
    # 먼저 기존 열의 리스트를 가져옴
    columns_list = data.columns.tolist()
    # 예측값을 데이터프레임으로 변환
    predictions_df = pd.DataFrame(predicted_disp_index1, columns=[f'disp_index{i+1}'])
    # 기존 데이터프레임과 예측값을 결합
    data_with_predictions = pd.concat([
        data.iloc[:, 0],  # 첫 번째 열
        predictions_df,    # 예측값 (두 번째 열로 들어감)
        data.iloc[:, 1:]  # 나머지 열들
    ], axis=1)

    # 결과를 CSV 파일로 저장
    data_with_predictions.to_csv(f'filehome/updated_preprocessed_yyeepp_with_predictions{i+1}.csv', index=False)

    # 저장 확인을 위한 출력
    print("예측값이 추가된 데이터 미리보기:")
    print(data_with_predictions.head())
    print("\n컬럼 순서 확인:")
    print(data_with_predictions.columns.tolist())

In [ ]:
import pandas as pd
import glob
import os

def merge_csv_files_horizontally():
    # 분할된 CSV 파일들을 찾습니다
    # csv_files = sorted(glob.glob('split_data_*.csv'), 
    #                   key=lambda x: int(x.split('_')[2].split('.')[0]))
    csv_files = ['filehome/updated_preprocessed_yyeepp_with_predictions1.csv','filehome/updated_preprocessed_yyeepp_with_predictions2.csv','filehome/updated_preprocessed_yyeepp_with_predictions3.csv','filehome/updated_preprocessed_yyeepp_with_predictions4.csv','filehome/updated_preprocessed_yyeepp_with_predictions5.csv']
    
    # 각 파일의 데이터를 저장할 리스트
    dfs = []
    
    # 각 CSV 파일을 읽어옵니다
    for file in csv_files:
        df = pd.read_csv(file)
        
        # 파일 이름 형식을 확인하고, 올바를 경우에만 처리
        try:
            file_num = file.split('_')[2].split('.')[0]
        except IndexError:
            print(f"파일 이름 형식이 올바르지 않습니다: {file}")
            continue
        
        # 컬럼 이름을 파일 번호를 포함하도록 변경
        new_columns = {col: f'{col}_{file_num}' for col in df.columns}
        df = df.rename(columns=new_columns)
        
        dfs.append(df)
    
    # 모든 데이터프레임의 행 수를 가장 긴 데이터프레임에 맞춤
    max_rows = max(len(df) for df in dfs)
    for i in range(len(dfs)):
        if len(dfs[i]) < max_rows:
            # 부족한 행을 NaN으로 채웁니다
            dfs[i] = dfs[i].reindex(range(max_rows))
    
    # 데이터프레임들을 수평으로 연결
    result = pd.concat(dfs, axis=1)
    
    # 결과를 새로운 CSV 파일로 저장
    result.to_csv('filehome/merged_data.csv', index=False)
    print(f'병합된 파일이 생성되었습니다. 총 {len(result)} 행, {len(result.columns)} 열')
    
    return result

# 함수 실행
merged_df = merge_csv_files_horizontally()


In [ ]:
loaded_model = joblib.load(f'BernoulliNB_best (1).joblib')
X_1 = pd.read_csv('filehome/merged_data.csv')
y_pred = loaded_model.predict(X_1)

